# 01 – Exploration du Dataset HAM10000

Ce notebook effectue l'analyse exploratoire du dataset HAM10000 :

* Vérification de la structure des dossiers
* Distribution des classes (visualisation)
* Affichage d'exemples d'images par classe
* Préparation des données pour l'entraînement

In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from PIL import Image

# Chemins de base (le notebook est dans ia_vision/notebooks/)
base_dir = Path("..")  # ia_vision/
raw_dir  = base_dir / "data" / "raw"

for sub in ["raw", "processed", "splits"]:
    path = base_dir / "data" / sub
    print(f"{sub}: {path.resolve()} — {'OK' if path.exists() else 'absent'}")

## Distribution des classes

In [ ]:
CLASSES = ["akiec", "bcc", "bkl", "df", "mel", "nv", "vasc"]
CLASS_LABELS = {
    "akiec": "Actinic keratoses",
    "bcc":   "Basal cell carcinoma",
    "bkl":   "Benign keratosis-like lesions",
    "df":    "Dermatofibroma",
    "mel":   "Melanoma",
    "nv":    "Melanocytic nevi",
    "vasc":  "Vascular lesions",
}

metadata_path = raw_dir / "HAM10000_metadata.csv"

if metadata_path.exists():
    df = pd.read_csv(metadata_path)
    print(f"Dataset : {len(df)} images, {df['dx'].nunique()} classes")
    print(df['dx'].value_counts())

    counts = df['dx'].value_counts().reindex(CLASSES)
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].bar(counts.index, counts.values, color=sns.color_palette("Set2", len(CLASSES)))
    axes[0].set_title("Distribution des classes (nombre d'images)")
    axes[0].set_xlabel("Classe")
    axes[0].set_ylabel("Nombre d'images")
    axes[0].tick_params(axis='x', rotation=30)
    axes[1].pie(
        counts.values,
        labels=[CLASS_LABELS[c] for c in CLASSES],
        autopct='%1.1f%%',
        colors=sns.color_palette("Set2", len(CLASSES)),
        startangle=140,
    )
    axes[1].set_title("Répartition (%)")
    plt.tight_layout()
    plt.show()
else:
    print("Fichier metadata introuvable – placez HAM10000_metadata.csv dans data/raw/")
    print("Affichage d'une distribution simulée (chiffres réels HAM10000).")
    simulated = {"akiec": 327, "bcc": 514, "bkl": 1099, "df": 115,
                 "mel": 1113, "nv": 6705, "vasc": 142}
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].bar(simulated.keys(), simulated.values(),
                color=sns.color_palette("Set2", 7))
    axes[0].set_title("Distribution HAM10000 (simulée)")
    axes[0].set_xlabel("Classe")
    axes[0].set_ylabel("Nombre d'images")
    axes[0].tick_params(axis='x', rotation=30)
    axes[1].pie(
        simulated.values(),
        labels=[CLASS_LABELS[c] for c in simulated],
        autopct='%1.1f%%',
        colors=sns.color_palette("Set2", 7),
        startangle=140,
    )
    axes[1].set_title("Répartition (%)")
    plt.tight_layout()
    plt.show()

## Exemples d'images par classe

In [ ]:
processed_dir = base_dir / "data" / "processed"

if processed_dir.exists():
    fig, axes = plt.subplots(1, len(CLASSES), figsize=(20, 3))
    for ax, cls in zip(axes, CLASSES):
        cls_dir = processed_dir / cls
        if cls_dir.exists():
            files = list(cls_dir.glob("*.jpg")) + list(cls_dir.glob("*.jpeg"))
            if files:
                img = Image.open(files[0])
                ax.imshow(img)
                ax.set_title(cls, fontsize=9)
                ax.axis('off')
                continue
        ax.text(0.5, 0.5, 'N/A', ha='center', va='center')
        ax.set_title(cls, fontsize=9)
        ax.axis('off')
    plt.suptitle("Un exemple par classe", y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("Dossier data/processed introuvable – lancez d'abord src/preprocess.py")

## Étapes suivantes

1. Placer les images et `HAM10000_metadata.csv` dans `data/raw/`.
2. Lancer `src/preprocess.py` pour redimensionner les images et créer les splits.
3. Entraîner le modèle avec `src/train.py`.
4. Consulter les résultats dans `notebooks/02_training_results.ipynb`.
5. Utiliser `src/predict.py` dans le backend pour les prédictions.